In [1]:
# !pip install torchmetrics

In [2]:
import torch 
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import h5py
import math
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from torch.utils.data import Dataset, random_split, DataLoader
from torchvision import transforms
import torch.optim as optim
from torchmetrics.classification import MulticlassAUROC, MulticlassAccuracy

In [3]:
# clearing cuda cache memory
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [4]:
os.listdir("../dataset")

['QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272',
 'QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272.test.snappy.parquet',
 'QCDToGGQQ_IMGjet_RH1all_jet0_run1_n47540',
 'QCDToGGQQ_IMGjet_RH1all_jet0_run1_n47540.test.snappy.parquet',
 'QCDToGGQQ_IMGjet_RH1all_jet0_run2_n55494',
 'QCDToGGQQ_IMGjet_RH1all_jet0_run2_n55494.test.snappy.parquet',
 'quark-gluon_data-set_n139306.hdf5',
 'SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5',
 'SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5']

In [5]:
# import dataset
electron_dataset = h5py.File("../dataset/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5","r")
electron_imgs=np.array(electron_dataset["X"])
electron_labels=np.array(electron_dataset["y"],dtype=np.int64)

photon_dataset = h5py.File("../dataset/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5","r")
photon_imgs=np.array(photon_dataset["X"])
photon_labels=np.array(photon_dataset["y"],dtype=np.int64)

In [6]:
# GoogLeNet
# Xception
# SENet

In [7]:
img_arrs = torch.Tensor(np.vstack((photon_imgs,electron_imgs)))
labels = torch.Tensor(np.hstack((photon_labels,electron_labels))).to(torch.int64)

In [8]:
class SingleElectronPhotonDataset(Dataset):
    def __init__(self,split_inx, transform=None,target_transform= None):
        self.img_arrs_split = img_arrs[split_inx]
        self.labels_split = labels[split_inx]
        self.transform = transform
        self.target_transform = target_transform
    def __len__(self):
        return self.labels_split.shape[0]
    def __getitem__(self,idx):
        image=self.img_arrs_split[idx,:,:,:]
        # changing the dim of image to channels, height, width by transposing the
        # original image tensor.
        image = image.permute(2,1,0)
        label = self.labels_split[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image,label

In [25]:
class ResidualUnit(nn.Module):
    def __init__(self,in_channels, out_channels, **kwargs):
        """ Constructor
        Args:
            isSkipLearnable: learnalbe weights are associated with skip connection.
        """
        super().__init__(**kwargs)
        strides = 1
        if in_channels == out_channels:
            strides = 1
            pad = "same"
        else:
            strides = 2
            pad = 1
        self.relu = nn.ReLU(inplace=True)
        self.main_layers = nn.ModuleList([
            nn.Conv2d(in_channels,out_channels,3,strides,padding=pad,bias=False),
            nn.BatchNorm2d(out_channels),
            self.relu,
            nn.Conv2d(out_channels,out_channels,3,stride=1,padding="same",bias=False),
            nn.BatchNorm2d(out_channels)
        ])
        
        self.skip_layers =[]
        if strides > 1 :
            self.skip_layers = nn.ModuleList([
                nn.Conv2d(in_channels,out_channels,1,strides,padding=0,bias=False),
                nn.BatchNorm2d(out_channels)
            ])
    def forward(self,x):
        Z = x 
        for layer in self.main_layers:
            Z = layer(Z)
        skip_z = x
        for layer in self.skip_layers:
            skip_z= layer(skip_z)
        return self.relu(Z + skip_z)  

class ResNet18(nn.Module):
    def __init__(self,num_classes):
        super(ResNet18, self).__init__()

        self.num_classes = num_classes
        self.relu = nn.ReLU(inplace=True)
        
        self.conv1 = nn.Conv2d(2,64,3,stride=2,padding=1,bias=False)
        self.bn1= nn.BatchNorm2d(64)
        self.max_pool = nn.MaxPool2d(1,1) 
        prev_filters = 64
        self.res_unit_list = nn.ModuleList([ResidualUnit(prev_filters,prev_filters)])
        for filters in [64]*1+[128]*2 + [256]*2 +[512]*2:
            self.res_unit_list.append(ResidualUnit(prev_filters,filters))
            prev_filters = filters 
#         self.fc = nn.LazyLinear(num_classes)
        self.fc = nn.Linear(512, num_classes)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.max_pool(x)
        for res_unit in self.res_unit_list:
            x = res_unit(x)
        print("here 18 shape: ",x.shape)
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return F.softmax(x,dim=1)
    
    def __str__(self):
        return "ResNet18"
    
    
class ResNet34(nn.Module):
    def __init__(self,num_classes):
        super(ResNet34, self).__init__()

        self.num_classes = num_classes
        self.relu = nn.ReLU(inplace=True)
        
        self.conv1 = nn.Conv2d(2,64,7,stride=2,padding=3,bias=False)
        self.bn1= nn.BatchNorm2d(64)
        self.max_pool = nn.MaxPool2d(1,1) 
        prev_filters = 64
        self.res_unit_list = nn.ModuleList([ResidualUnit(prev_filters,prev_filters)])
        for filters in [64]*2+[128]*4 + [256]*6 +[512]*3:
            self.res_unit_list.append(ResidualUnit(prev_filters,filters))
            prev_filters = filters 
#         self.fc = nn.LazyLinear(num_classes)
        self.fc = nn.Linear(512, num_classes)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.max_pool(x)
        for res_unit in self.res_unit_list:
            x = res_unit(x)
        print("here shape: ",x.shape)
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return F.softmax(x,dim=1)
    
    def __str__(self):
        return "ResNet34"

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else torch.device("cpu"))

In [26]:
model = ResNet18(num_classes=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
multicls_criterion = torch.nn.CrossEntropyLoss()

epochs = 25

In [12]:
preprocess = transforms.Compose([
#     transforms.Resize(224),
    transforms.Resize(128), # multiply of 16
    transforms.Normalize(mean=[0.5, 0.5], std=[0.5, 0.5]),
])

train_inx, valid_inx, test_inx = random_split(range(labels.shape[0]),[0.7,0.2,0.1],generator=torch.Generator()
                                            .manual_seed(42))

train_data = SingleElectronPhotonDataset(split_inx=train_inx,transform = preprocess)
valid_data = SingleElectronPhotonDataset(split_inx=valid_inx,transform = preprocess)
test_data = SingleElectronPhotonDataset(split_inx=test_inx,transform = preprocess)
# dataset = SingleElectronPhotonDataset()

train_dataloader = DataLoader(train_data,batch_size = 64, shuffle = True)
valid_dataloader = DataLoader(valid_data,batch_size = 64, shuffle = True)
test_dataloader = DataLoader(test_data,batch_size = 64, shuffle = True)

In [13]:
def train(model, device, loader, optimizer):
    model.train()

    loss_accum = 0
    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        output = model(inputs)
        loss= 0 
        optimizer.zero_grad()
        loss += multicls_criterion(output, labels)
        loss.backward()
        optimizer.step()

        loss_accum += loss.item()

    print('Average training loss: {}'.format(loss_accum / (step + 1))) 

In [14]:
def evaluate(model, device, loader,evaluator= "roauc"):
    model.eval()
    
    preds_list = []
    target_list = []
    for step, batch in enumerate(loader):
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            output = model(inputs)
            preds_list.extend(output.tolist())
        target_list += batch[1].tolist()
    if evaluator == "roauc":   
        metric = MulticlassAUROC(num_classes=2, average="macro", thresholds=None)
    if evaluator == "acc":
        metric = MulticlassAccuracy(num_classes=2, average="macro")
    # print("AUC-ROC metric score : ",metric(torch.Tensor(preds_list),torch.Tensor(target_list)).item())
    return metric(torch.Tensor(preds_list),torch.Tensor(target_list).to(torch.int64)).item()

In [15]:
checkpoints_path = "../models"
checkpoints = os.listdir(checkpoints_path)
checkpoint_path = list(filter(lambda i : str(model) in i, checkpoints))

In [27]:
train_curves = []
valid_curves = []

starting_epoch = 1
if len(checkpoint_path)>0:
    checkpoint = torch.load(f"{checkpoints_path}/{checkpoint_path[0]}")
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    starting_epoch = checkpoint['epoch']+1

for epoch in range(starting_epoch, epochs + 1):
    print("=====Epoch {}".format(epoch))
    print('Training...')
    train(model, device, train_dataloader, optimizer)
    
    print("Evaluating...")
    train_perf_roauc = evaluate(model,device,train_dataloader)
    valid_perf_roauc = evaluate(model,device,valid_dataloader)
    test_perf_roauc = evaluate(model,device,test_dataloader)
    train_perf_acc = evaluate(model,device,train_dataloader, evaluator = "acc")
    valid_perf_acc = evaluate(model,device,valid_dataloader,evaluator = "acc")
    test_perf_acc = evaluate(model,device,test_dataloader,evaluator = "acc")
    
    train_curves.append([train_perf_acc,train_perf_roauc])
    valid_curves.append([valid_perf_acc,valid_perf_roauc])
    
    print('ROAUC scores: ',{'Train': train_perf_roauc, 'Validation': valid_perf_roauc, "Test": test_perf_roauc}, '\nAccuracy scores: ',
         {'Train': train_perf_acc, 'Validation': valid_perf_acc, "Test": test_perf_acc})
    
    # save checkpoint of current epoch
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, f"{checkpoints_path}/{str(model)}-{epoch}.pt")
    
    # delete checkpoint of previous epoch
    if epoch>1:
        os.remove(f"{checkpoints_path}/{str(model)}-{epoch-1}.pt")

print('\nFinished training!')
print('\nROAUC Test score: {}'.format(evaluate(model,device,test_dataloader)))

=====Epoch 1
Training...


Iteration:   0%|          | 0/5447 [00:00<?, ?it/s]

here 18 shape:  torch.Size([64, 512, 8, 8])


Iteration:   0%|          | 2/5447 [00:00<31:04,  2.92it/s]

here 18 shape:  torch.Size([64, 512, 8, 8])


...

Iteration:   0%|          | 18/5447 [00:04<18:31,  4.89it/s]

here 18 shape:  torch.Size([64, 512, 8, 8])


Iteration:   0%|          | 18/5447 [00:04<21:31,  4.20it/s]

here 18 shape:  torch.Size([64, 512, 8, 8])


KeyboardInterrupt: 

In [36]:
arr = torch.rand(64,512,1,1)

In [37]:
arr_ = F.interpolate(arr, scale_factor=8)

In [38]:
arr_.shape

torch.Size([64, 512, 8, 8])

In [46]:
unpool = nn.MaxUnpool2d(1,1)
pool = nn.MaxPool2d(1,1)

In [44]:
arr_.shape

torch.Size([64, 512, 8, 8])

In [48]:
pool(arr_).shape

torch.Size([64, 512, 8, 8])

In [59]:
pool = nn.MaxPool2d(3, stride=2, padding = 1, return_indices=True)
unpool = nn.MaxUnpool2d(2, stride=2)
input = torch.tensor(    [[[[ 1.,  0.,  3.,  4.,1.,  0.,  3.,  4.],
                            [ 6.,  2.,  7.,  8., 6.,  2.,  7.,  8.],
                            [ 9., 10., 11., 12., 6.,  2.,  7.,  8.],
                            [13., 14., 15., 16., 6.,  2.,  7.,  8.],
                            [13., 14., 15., 16., 6.,  2.,  7.,  8.],
                            [13., 14., 15., 16., 6.,  2.,  7.,  8.]]]])
output, indices = pool(input)
print(indices)
print(output)
unpool(output, indices)

tensor([[[[ 8, 11, 11, 15],
          [25, 27, 27, 15],
          [25, 27, 27, 31]]]])
tensor([[[[ 6.,  8.,  8.,  8.],
          [14., 16., 16.,  8.],
          [14., 16., 16.,  8.]]]])


tensor([[[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 6.,  0.,  0.,  8.,  0.,  0.,  0.,  8.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0., 14.,  0., 16.,  0.,  0.,  0.,  8.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]]])

In [ ]:

# Now using output_size to resolve an ambiguous size for the inverse

output, indices = pool(input)
# This call will not work without specifying output_size
unpool(output, indices, output_size=input.size())